In [ ]:
# default_exp core

In [1]:
%load_ext lab_black

# Core functions

> API details.

In [2]:
# hide
from nbdev.showdoc import *
from fastcore.test import *
import torch

In [3]:
# export
def hello():
    print("torch.cuda.is_available():", torch.cuda.is_available())
    return "hello"

In [4]:
test_eq("hello", hello())

torch.cuda.is_available(): False


In [6]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
